In [11]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd

import rt_analysis as rt
import rt_analysis2 as rt2
import importlib

In [12]:
lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')
stop_times = pd.read_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')
stops = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_stops.parquet')

In [13]:
trips_positions_joined = (lbt_trips
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id', 'calitp_url_number'])
                       )

In [14]:
trips_positions_joined.vehicle_timestamp.max()

Timestamp('2021-12-01 11:15:59')

In [15]:
trips_positions_joined.vehicle_timestamp.min()

Timestamp('2021-11-29 15:57:13')

In [16]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [17]:
lbt_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_routelines.parquet')

In [18]:
trips_positions_joined = trips_positions_joined >> filter(_.vehicle_timestamp.apply(lambda x: x.day == 30))

In [19]:
trips_positions_joined.head(3) ## 'raw' vehicle positions data joined with trip info

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
285,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:50:38,POINT (174524.511 -468602.082)
286,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:50:38,POINT (174524.511 -468602.082)
287,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:51:08,POINT (174524.511 -468602.082)


In [20]:
example_trip = (trips_positions_joined
 >> filter(_.trip_key == 1553149201202261125)
 >> arrange(_.vehicle_timestamp)
)

In [21]:
example_trip.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
327364,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:02:37,POINT (174550.467 -468602.675)
327365,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:02:37,POINT (174550.467 -468602.675)
327366,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:03:07,POINT (174550.467 -468602.675)


In [24]:
importlib.reload(rt)
importlib.reload(rt2)

<module 'rt_analysis2' from '/home/jovyan/data-analyses/rt_delay/rt_analysis2.py'>

In [25]:
example_vp_trip2 = rt2.VehiclePositionsInterpolator(example_trip, lbt_routelines)

sc_called


In [26]:
example_vp_trip2.position_gdf.iloc[47:50] ## example of non-increasing route distance

,calitp_url_number,calitp_extracted_at,calitp_deleted_at,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
47,0,2021-09-16,2099-01-01,2021-11-30 17:46:29,33.791615,-118.22247,2021-11-30 17:46:38,POINT (164640.398 -467735.385),11131.857239,2021-11-30 17:45:29,11121.586008,60.0,10.271231,True,0.171187
48,0,2021-09-16,2099-01-01,2021-11-30 17:47:30,33.791607,-118.22247,2021-11-30 17:47:38,POINT (164640.415 -467736.272),11130.970070,2021-11-30 17:46:29,11131.857239,61.0,-0.887169,False,-0.014544
49,0,2021-09-16,2099-01-01,2021-11-30 17:48:30,33.791580,-118.22246,2021-11-30 17:48:38,POINT (164641.397 -467739.248),11130.303860,2021-11-30 17:47:30,11130.970070,60.0,-0.666210,False,-0.011103


In [27]:
example_vp_trip2.cleaned_positions.tail(5) ## cleaned gdf, and likely base for a future view

,calitp_url_number,calitp_extracted_at,calitp_deleted_at,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
327484,0,2021-09-16,2099-01-01,2021-11-30 17:42:30,33.789967,-118.214676,2021-11-30 17:42:38,POINT (165365.659 -467904.545),10112.388594,2021-11-30 17:41:30,9865.353045,60.0,247.035549,True,4.117259
327487,0,2021-09-16,2099-01-01,2021-11-30 17:43:30,33.789986,-118.216340,2021-11-30 17:43:38,POINT (165211.510 -467905.339),10266.432340,2021-11-30 17:42:30,10112.388594,60.0,154.043746,True,2.567396
327490,0,2021-09-16,2099-01-01,2021-11-30 17:44:29,33.789950,-118.217180,2021-11-30 17:44:38,POINT (165133.789 -467910.794),10344.319040,2021-11-30 17:43:30,10266.432340,59.0,77.886700,True,1.320114
327493,0,2021-09-16,2099-01-01,2021-11-30 17:45:29,33.791523,-118.223790,2021-11-30 17:45:38,POINT (164518.341 -467747.877),11121.586008,2021-11-30 17:44:29,10344.319040,60.0,777.266967,True,12.954449
327496,0,2021-09-16,2099-01-01,2021-11-30 17:46:29,33.791615,-118.222470,2021-11-30 17:46:38,POINT (164640.398 -467735.385),11131.857239,2021-11-30 17:45:29,11121.586008,60.0,10.271231,True,0.171187


In [28]:
example_vp_trip2.trip_key

1553149201202261125

In [29]:
## estimate of when the bus reached 1000 meters along the shape, based on nearest 2 vehicle position reports
example_vp_trip2.time_at_position(700)

Timestamp('2021-11-30 17:04:17.773260')

In [30]:
# example_vp_trip2.detailed_speed_map()

### Focusing an analysis

In [31]:
example_trips = (trips_positions_joined
 >> filter(_.route_id == '175', _.direction_id == '0') ## towards Villages at Cabrillo
 >> group_by(_.trip_key)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 >> head(6) ## afternoon/evening
)
example_trips

,trip_key,max_time
4,-5374575435373645071,2021-11-30 07:14:16
12,3082123411846948636,2021-11-30 07:54:17
0,-8296488155279914403,2021-11-30 08:38:01
1,-7508921904019176003,2021-11-30 09:15:30
14,4908653065062143760,2021-11-30 09:54:22
5,-4201736542695259394,2021-11-30 10:35:43


In [32]:
%%capture
## suppress warnings from location data inaccuracies
## generate test analysis with a small subset of LBT trips on a particular route, time, direction
test_analysis = rt.RtAnalysis(trips_positions_joined, stop_times, stops, lbt_routelines, example_trips.trip_key)

In [33]:
%%capture
## suppress warnings from start/end of routes
all_delays = test_analysis.generate_delay_view()

In [34]:
all_summary = test_analysis.generate_delay_summary()

In [35]:
all_summary.head(5)

,stop_id,geometry,stop_sequence,avg_delay,max_delay
0,1027,POINT (174124.462 -468604.478),2,0 days 00:01:26.446291666,0 days 00:04:11.371129
1,0540,POINT (173921.407 -468897.107),3,0 days 00:01:07.972914400,0 days 00:04:28.865438
2,0542,POINT (173328.363 -468647.513),4,0 days 00:00:39.873642666,0 days 00:03:58.590881
3,0543,POINT (173108.288 -468477.715),5,0 days 00:00:37.971737666,0 days 00:03:32.980943
4,0544,POINT (173001.551 -468534.279),6,0 days 00:01:56.187369166,0 days 00:03:37.006580


In [36]:
all_delays.head(5)

,trip_key,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,service_date,geometry
144,-7508921904019176003,2021-11-30 08:21:41,2021-11-30 08:25:52.371129,0 days 00:04:11.371129,1027,9288597,1750012,0,Westbound,2,175,501.040761,2021-11-30,POINT (174124.462 -468604.478)
148,-4201736542695259394,2021-11-30 09:44:55,2021-11-30 09:44:21.605429,-1 days +23:59:26.605429,1027,9288601,1750012,0,Westbound,2,175,501.040761,2021-11-30,POINT (174124.462 -468604.478)
145,3082123411846948636,2021-11-30 07:02:41,2021-11-30 07:03:22.362317,0 days 00:00:41.362317,1027,9288593,1750012,0,Westbound,2,175,501.040761,2021-11-30,POINT (174124.462 -468604.478)
88,-8296488155279914403,2021-11-30 07:42:20,2021-11-30 07:44:30.632731,0 days 00:02:10.632731,0540,9288595,1750012,0,Westbound,3,175,991.109291,2021-11-30,POINT (173921.407 -468897.107)
89,-7508921904019176003,2021-11-30 08:23:20,2021-11-30 08:27:48.865438,0 days 00:04:28.865438,0540,9288597,1750012,0,Westbound,3,175,991.109291,2021-11-30,POINT (173921.407 -468897.107)


In [80]:
test_analysis.map_segment_speeds(how = 'low_speeds')

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


1750012 (27, 14)


In [81]:
test_analysis.segment_speed_view.head(3)

,stop_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,service_date,geometry,seconds_from_last,last_loc,meters_from_last,speed_from_last,speed_mph
0,0540,1750012,0,Westbound,3,175,991.0,2021-11-30,"LINESTRING (174124.618 -468612.350, 174123.692...",116.0,501.040761,490.068530,4.224729,10.5
1,0542,1750012,0,Westbound,4,175,1666.0,2021-11-30,"LINESTRING (173927.149 -468896.993, 173927.149...",76.0,991.109291,675.293145,8.885436,18.9
2,0543,1750012,0,Westbound,5,175,1955.0,2021-11-30,"LINESTRING (173323.344 -468652.566, 173323.272...",56.0,1666.402436,288.544820,5.152586,7.7


In [82]:
test_analysis.route_coverage_summary() ## for each trip, how far does RT actually cover?

,trip_key,trip_id,shape_id,direction_id,min_meters,min_stop,max_meters,max_stop
0,-8296488155279914403,9288595,1750012,0,991.109291,3,10745.097772,28
1,-7508921904019176003,9288597,1750012,0,501.040761,2,11480.641185,29
2,-5374575435373645071,9288591,1750012,0,1666.402436,4,10745.097772,28
3,-4201736542695259394,9288601,1750012,0,501.040761,2,10745.097772,28
4,3082123411846948636,9288593,1750012,0,501.040761,2,10745.097772,28
5,4908653065062143760,9288599,1750012,0,991.109291,3,11480.641185,29


In [47]:
test_analysis.map_stop_delays()

In [48]:
one_trip = lbt_trips >> filter(_.trip_key == 3082123411846948636)
one_trip = one_trip.drop(columns=['calitp_extracted_at', 'calitp_deleted_at'])

trip_st_gdf = test_analysis.st_geo >> inner_join(_, one_trip,
                                           on = ['calitp_itp_id', 'calitp_url_number', 'trip_id'])

In [49]:
schedule = rt2.ScheduleInterpolator(trip_st_gdf = trip_st_gdf, shape_gdf = lbt_routelines)

In [50]:
schedule.cleaned_positions.head(3)

,stop_id,stop_lat,stop_lon,stop_name,geometry,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,...,shape_dist_traveled,timepoint,calitp_extracted_at,arrival_dt,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,speed_from_last
19,1026,33.782069,-118.114850,BEACH DR & W. CAMPUS N,POINT (174628.272 -468601.239),1,07:01:00,07:01:00,None,0,...,0.0,1,2021-12-17,2021-11-30 07:01:00,0.000000,2021-11-30 07:08:56,2366.576250,85924.0,-2366.576250,-0.027543
24,1027,33.782130,-118.120288,BEACH DR. & DORMS N,POINT (174124.462 -468604.478),2,07:02:41,07:02:41,None,0,...,0.311,0,2021-12-17,2021-11-30 07:02:41,501.040761,2021-11-30 07:20:19,6170.938186,85342.0,-5669.897425,-0.066437
14,0540,33.779528,-118.122542,BELLFLOWER & ANAHEIM RD NW,POINT (173921.407 -468897.107),3,07:04:20,07:04:20,None,0,...,0.616,0,2021-12-17,2021-11-30 07:04:20,991.109291,2021-11-30 07:32:00,10230.820319,84740.0,-9239.711028,-0.109036


In [56]:
one_trip = trips_positions_joined >> filter(_.trip_key == 3082123411846948636)

In [87]:
rt_trip = rt2.VehiclePositionsInterpolator(one_trip, lbt_routelines)

sc_called
(159, 8)


In [88]:
# rt.detailed_speed_map()

In [89]:
schedule.time_at_position(3300)

Timestamp('2021-11-30 07:11:54.133086')

In [90]:
rt_trip.time_at_position(3300)

Timestamp('2021-11-30 07:13:46.822929')

In [91]:
def delay_at_position(schedule, rt, pos):
    schedule_ts = schedule.time_at_position(pos)
    rt_ts = rt.time_at_position(pos)
    return rt_ts - schedule_ts

In [92]:
delay_at_position(schedule, rt_trip, 3000)

Timedelta('0 days 00:02:12.713118')